# <center>Battle of the Neighborhoods  <br>-<br>What to Eat in a Financial Metropolis?</center>

## The Business Idea

Global financial centers are some of the most wealthy regions. They are spacially very concentrated and a hot spot for luxury products. In the aftermath of the coronavirus pandemic, thousands of restaurants shut down for good, even in the busy financial centers. But ones downfall is the others chance and people always need places to go and eat. New restaurants are needed! The idea is to open a restaurant in Frankfurt am Main (short: FFM), Germany. As a restaurant, especially nowadays, you want to minimize risk. So this is part of a whole market study an investor could rely on for his decisions. So what kind of restaurant? What does the international financial elite love to eat? Is there a prevalent type of restaurant between those cities? Let's find out!

## The Data 

Different cultures, different food, right? But do mondane people favor their native kitchen? Or do they all agree on one trendy type of food all over the world? To get a better overview, several cities with big financial centers will be analyzed: New York, Toronto, Tokyo, London and Frankfurt. The data used will be provided by <a href = "url">foursquare.com</a>, automatically downloaded via their API. The focus will lie on restaurants only, excluding cafes, bars, bistros and so on.

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

<br>Adding "&query=food" to the Foursquare API url to only get food-related venues. This way the search radius can be expanded without hitting the limit of 100 venues per city. The request and subsequent data frame building is looped over all 5 cities.

In [2]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# list of cities and their financial centers as coordinates
cities = ["New York", "Toronto", "Tokyo", "London", "Frankfurt"]
latitudes = [40.7077426113486, 43.64803026178278, 35.679822325779995, 51.515259192193064, 50.11081120228777]
longitudes = [-74.01157215146745, -79.3816696346114, 139.77014548424125, -0.08039578093893954, 8.67324827528688]

# Foursquare API credentials
CLIENT_ID = 'T0ETMOBTMRGKDWDG5VBSUFRUVWYJDTLHMLZHN2APZ545NNGR' # your Foursquare ID
CLIENT_SECRET = 'T03SSSFDVCM1JFVTX4YJLGS2IJPYD2AMRNWAR4DAXF2RFMZA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

# a list to store all data frames in
all_df = list()

# loop over all 5 cities
for i in range(len(cities)):
# create URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=food'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitudes[i], 
        longitudes[i], 
        radius, 
        LIMIT)

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues['city'] = cities[i]
# add city name to column
    all_df.append(nearby_venues)
    
all_df[0].head()

<ipython-input-2-5efed386b48f>:42: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng,city
0,Crown Shy,Restaurant,40.706187,-74.007490,New York
1,sweetgreen,Salad Place,40.705626,-74.008282,New York
2,Pisillo Italian Panini,Sandwich Place,40.710530,-74.007526,New York
3,Hudson Eats,Food Court,40.712666,-74.015901,New York
4,Los Tacos No. 1,Taco Place,40.714267,-74.008756,New York


(Note: this is only one of 5 data frames. The visualization of all comes with a map)

<br>Although bistros and cafes also serve food, for simplicity, we use restaurant-type categories only.

In [3]:
# list restaurants data frames
restaurants = list()
for i in range(5):
    restaurants.append(all_df[i][all_df[i]['categories'].str.match(".*Restaurant*.")])
restaurants[1].head()

,name,categories,lat,lng,city
0,Canoe,Restaurant,43.647452,-79.381320,Toronto
1,Pai,Thai Restaurant,43.647923,-79.388579,Toronto
2,Richmond Station,American Restaurant,43.651569,-79.379266,Toronto
3,Byblos Toronto,Mediterranean Restaurant,43.647615,-79.388381,Toronto
4,The Keg Steakhouse + Bar - York Street,Restaurant,43.649987,-79.384103,Toronto


Some restaurants don't have a specification for their kitchen. These will be treated as "miscellaneous" category.

In [4]:
for i in range(5):
    print("Number of restaurants in",cities[i],":  ",len(restaurants[i]))
    print("Misc. Restaurants:", sum(restaurants[i]['categories'].str.match("Restaurant")) )

Number of restaurants in New York :   56
Misc. Restaurants: 1
Number of restaurants in Toronto :   53
Misc. Restaurants: 8
Number of restaurants in Tokyo :   66
Misc. Restaurants: 1
Number of restaurants in London :   55
Misc. Restaurants: 7
Number of restaurants in Frankfurt :   48
Misc. Restaurants: 5


In [5]:
# renaming "Restaurants" to "Miscellaneous Restaurant"
for i in range(5):
    restaurants[i].loc[:]['categories'] = restaurants[i].loc[:]['categories'].replace('Restaurant','Miscellaneous Restaurant')
restaurants[1].head()

,name,categories,lat,lng,city
0,Canoe,Miscellaneous Restaurant,43.647452,-79.381320,Toronto
1,Pai,Thai Restaurant,43.647923,-79.388579,Toronto
2,Richmond Station,American Restaurant,43.651569,-79.379266,Toronto
3,Byblos Toronto,Mediterranean Restaurant,43.647615,-79.388381,Toronto
4,The Keg Steakhouse + Bar - York Street,Miscellaneous Restaurant,43.649987,-79.384103,Toronto


Finally, a map of the restaurants (Folium maps are not depicted when opened on GitHub, but you can run it on your own JupyterLab if u like)

In [8]:
map_all = folium.Map(location=[0, 0], zoom_start=2)

# add markers to map
for i in range(5):
    for lat, lng, name, category in zip(restaurants[i]['lat'], restaurants[i]['lng'], restaurants[i]['name'], restaurants[i]['categories']):
        label = '{}, {}'.format(name, category)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_all)  
    
map_all